In [6]:
import shutil
import os
import torchaudio
import librosa
import torch
import pickle as pkl
import audiofile
from soundfile import LibsndfileError
from speechbrain.inference.classifiers import EncoderClassifier

# Clear the model cache
cache_dir = "C:/Users/Adria/.cache/huggingface/hub/models--speechbrain--lang-id-voxlingua107-ecapa"
if os.path.exists(cache_dir):
    shutil.rmtree(cache_dir)

# Also remove the tmp directory
if os.path.exists("tmp"):
    shutil.rmtree("tmp")

# Then try loading the model again
from speechbrain.inference.classifiers import EncoderClassifier
language_id = EncoderClassifier.from_hparams(source="speechbrain/lang-id-voxlingua107-ecapa", savedir="tmp")


In [7]:
 # to be run on top of the 'uploads' directory containing the audio files
path = os.path.join(os.getcwd(), "uploads")
files = os.listdir(path)
supported_audio_formats = ["wav","mp3"]
files = [file for file in files if file[-3:] in supported_audio_formats]

In [8]:
path, files

('c:\\Users\\Adria\\dev\\blindwikiapp\\notebooks\\uploads',
 ['acoruna_barco_m67780_a84524_audio_converted.mp3',
  'barcelona_Ovvero_m68255_a84999_audio_converted.mp3',
  'berlin_Dirk_m67384_a84128_audio_converted.mp3'])

In [9]:
file_ends = set()
for file in files:
    file_ends.add(file[-3:])
file_ends

{'mp3'}

In [10]:
# Use the path variable you already defined
file_path = os.path.join(path, "barcelona_Ovvero_m68255_a84999_audio_converted.mp3")
os.path.exists(file_path)
signal, sampling_rate = audiofile.read(file_path)
signal

array([ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00, ...,
       -9.4561778e-07, -6.0664206e-07, -3.3898604e-07],
      shape=(254016,), dtype=float32)

In [20]:
predictions = []
corrupted_files = []
# Create tmp directory if it doesn't exist
data_dir = os.path.join(os.getcwd(), "data")
if not os.path.exists(data_dir):
    os.makedirs(data_dir)

# Your existing code
predictions_file = "predictions_list.pkl"
i = 0
for file in files[1:]:
    #print(file)
    i+=1
    try:
        signal, sr = audiofile.read(os.path.join(path, file))
    except LibsndfileError as e:
        print("Corrupt file",file)
        print(e)
        corrupted_files.append(file)
        continue
    prediction = language_id.classify_batch(torch.tensor(signal))
    predictions.append(prediction[3][0].split(":")[1].strip())
    if i % 100 == 0:
        print(f"Iteration {i} completed reaching update...")
        with open(os.path.join(data_dir, predictions_file),"wb") as f:
            pkl.dump(predictions,f)

with open(os.path.join(data_dir,predictions_file),"wb") as f:
    pkl.dump(predictions,f)